<a href="https://colab.research.google.com/github/cydal/Weather-App/blob/main/taikai_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import requests
import pandas as pd
import urllib.parse

In [112]:
dataset = "era5_volumetric_soil_water_layer_1-hourly"
dataset = "era5_land_precip-hourly"
dataset = "cpcc_temp_max-daily"
#dataset = "era5_wind_100m_u-hourly"
lat, lon = 5.68999972, -0.20985164

In [41]:
query = f"https://api.dclimate.net/apiv2/grid-history/{dataset}/{lat}_{lon}?also_return_metadata=false&use_imperial_units=true&also_return_snapped_coordinates=true&convert_to_local_time=true"
response = requests.get(query)

In [113]:
def get_data(dataset, lat, lon):
  query = f"https://api.dclimate.net/apiv2/grid-history/{dataset}/{lat}_{lon}?also_return_metadata=false&use_imperial_units=true&also_return_snapped_coordinates=true&convert_to_local_time=true"
  response = requests.get(query)

  print(response.status_code)
  if response.status_code != 200:
    return("Error: Check location & dataset")

  data = response.json()
  first = list(data['data'].keys())[0]
  last = list(data['data'].keys())[-1]

  key, value = [], []
  for k in data['data']:
    key.append(k)
    # remove text 
    if data['data'][k]:
      value.append(data['data'][k].split(" ")[0])
    else:
      value.append(data['data'][k])

  df = pd.DataFrame({
                    "date_time": key,
                    "value": value
  })
  df["date_time"] = pd.to_datetime(df["date_time"])


  return(df)

In [114]:
temp = get_data(dataset, lat, lon)

200


In [ ]:
temp

In [ ]:
temp["value"] # precip - '0.0002637795275590551 inch', temp_min - '67.676 deg_F', wind_v - '10.580708661417324 mi / h'

In [7]:
pd.to_datetime("2021-07-28 18:00:00").hour

18

In [10]:
address = 'Accra, Ghana'
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])

5.5571096
-0.2012376


In [14]:
lat, lon = response[0]["lat"], response[0]["lon"]

In [ ]:
Uyo, NG: January 18, 2021
Pressure: 1000.7 mb
Temperature: 29.5 C - cpcc_temp_min-daily & cpcc_temp_max-daily
Humidity: 77%
Condition: Cloudy
Wind: WSW at 5 m/s - era5_wind_100m_u-hourly & era5_wind_100m_v-hourly
Chance of rain: 80% - era5_land_precip-hourly

In [ ]:
def calc_wind(u, v):
  ws = np.sqrt(u**2 + v*82)
  wd = np.mod(180+ np.rad2deg(np.arctan2(u, v)), 360)
  return([ws, wd])

In [52]:
datasets = ["era5_wind_100m_u-hourly", "era5_wind_100m_v-hourly", "era5_land_precip-hourly", 
            "era5_volumetric_soil_water_layer_1-hourly", "cpcc_temp_min-daily", "cpcc_temp_max-daily"]

In [69]:
all_data = []
for dataset in datasets:
  all_data.append(get_data(dataset, lat, lon))


def get_all_data(datasets, , lat, lon):
  all_data = []

  for dataset in datasets:
    all_data.append(get_data(dataset, lat, lon))
  return(all_data)

200
200
200
200
200
200


In [71]:
all_data[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373078 entries, 0 to 373077
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   date_time  373078 non-null  object
 1   value      373078 non-null  object
dtypes: object(2)
memory usage: 5.7+ MB


In [94]:
temp

,date_time,value
0,1979-01-01 00:00:00+00:00,5.771295633500358
1,1979-01-01 01:00:00+00:00,6.598962061560488
2,1979-01-01 02:00:00+00:00,6.710808876163208
3,1979-01-01 03:00:00+00:00,6.710808876163208
4,1979-01-01 04:00:00+00:00,6.598962061560488
...,...,...
373073,2021-07-23 17:00:00+00:00,5.435755189692198
373074,2021-07-23 18:00:00+00:00,5.144953471725126
373075,2021-07-23 19:00:00+00:00,5.1002147458840374
373076,2021-07-23 20:00:00+00:00,5.793664996420902


In [133]:
new_temp = temp.copy()

In [135]:
new_temp.set_index("date_time")

,value
date_time,
1979-01-01,89.474
1979-01-02,89.97800000000001
1979-01-03,87.872
1979-01-04,85.19
1979-01-05,83.22800000000001
...,...
2021-07-22,84.77600000000001
2021-07-23,86.756
2021-07-24,86.95400000000001


In [ ]:
st.line_chart(new_temp[-14])

In [99]:
pd.to_datetime("2021-07-28 21:23:00").replace(second=0, microsecond=0, minute=0) #+timedelta(hours=t.minute//30)

Timestamp('2021-07-28 21:00:00')

In [126]:
query = "2021-07-10 21:23:00"

In [132]:
temp["value"][temp["date_time"] == pd.to_datetime(query).replace(second=0, microsecond=0, minute=0, hour=0)] # .dt.tz_convert(None)

15531    85.154
Name: value, dtype: object

In [ ]:
temp["value"][temp["date_time"]]

In [130]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15548 entries, 0 to 15547
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_time  15548 non-null  datetime64[ns]
 1   value      15524 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 243.1+ KB


Rain: Precipitation in the form of liquid water drops that have diameters greater than 0.5 millimeter; if less than 0.5 millimeter, it is drizzle. Rain and drizzle are the only forms of liquid precipitation. Rain is classified as light, meaning rain falling at a rate between a trace and 0.10 inch per hour; moderate, 0.11 to 0.30 inch per hour; heavy, more than 0.30 inch per hour. Drizzle is classified as light, falling at a rate from a trace to 0.01 inch per hour; moderate, 0.01 to 0.02 inch per hour; heavy, greater than 0.02 inch per hour.